In [1]:
%matplotlib inline

In [10]:
import pandas as pd
import numpy as np
import json

In [5]:
data = pd.read_csv('data/question_clean.csv')
data.head()
question = data.Question.to_numpy()

In [20]:
data = pd.read_csv('../crawling/data/total_google_play.csv', header=None)
data.head()

,0,1,2,3,4
0,invaders.os.galaxy.space.shooter.attack.classic,양희선,2019년 10월 12일,광고를 열 수 없다고 계속 진행되서 게임을 다시 지우고 깔았더니 3개월전 상태로 돌...,1
1,invaders.os.galaxy.space.shooter.attack.classic,이한란,2019년 10월 20일,대체 한달이 넘은거 갔네요.. 광고를 이용할수가 없다고 하고 게임시 승리점수도 적립...,1
2,invaders.os.galaxy.space.shooter.attack.classic,jinil kim,2019년 10월 16일,정말 재밌는 슛팅 게임입니다. 어렸을적 오락실 생각도 나고... 여튼 잠시나마 스트...,4
3,invaders.os.galaxy.space.shooter.attack.classic,Bum Chan Park,2019년 10월 9일,광고보기 안되는 오류 좀 못고치나요? 하루이틀도 아니고 아님 없애든가 이건뭐 열받아...,1
4,invaders.os.galaxy.space.shooter.attack.classic,이맹구,2019년 10월 9일,2틀전부터 팀에 접속도 안되고 pvp도 안되고 광고 동영상도 안되고 지우고 깔아도 ...,1


In [21]:
reviews = data.sample(n=len(question))[3].to_numpy()
len(reviews)

1305

In [14]:
del data

In [22]:
size = len(question)
labels = ['question'] * size + ['review'] * size
labels[:5]

['question', 'question', 'question', 'question', 'question']

In [24]:
print(len(question), len(reviews), len(labels))

1305 1305 2610


In [25]:
sample = pd.DataFrame()
sample['content'] = np.concatenate([question, reviews]) 
sample['label'] = labels
sample.head()

,content,label
0,블루스택 프로그램으로 좀비고 라는 게임을 하는데요 컨트롤러 조작이 원래는 wasd...,question
1,지금 데스티니아 깔고 실행하려하는데 안되네요. T스토어로 뭘 하라는데 t스토어 자체...,question
2,클랜시 오브클랜을 키면 화면이 왜 이러는 건가요? 해결법 부탁 드립니다.,question
3,게임제목좀 알려주세요모바일게임인지 궁금하네요,question
4,루팅 된 폰으로 파일안에까지 들여다 볼 수 있는 어플있잖아요 루트익스플로러같은..혹...,question


In [30]:
sample = sample.sample(frac=1)
sample.head()

,content,label
57,자꾸 사진처럼떠요 저장공간은 16GB중에 8GB정도밖에 안썼어요어떻게 해야 깔릴까요...,question
988,제 핸드폰게임을 컴으로 할 수 있나요 녹스 는 알겠는데 제 폰에 현질한 겜이 하나있어서요,question
698,어느순간부터 모바일겜이 렉이 심해졌네요. 폰은 노트9이구요 바꾼지는 1년조금 넘었습...,question
433,결제를 하다가 실수로 저도 모르게 다른 저도모르는 것들이 결제가 되었습니다제가 원래...,question
1943,간단하고 잼나고 귀엽고,review


In [31]:
sample.to_csv('data/sample.csv', index=False, header=False)

In [32]:
from collections import Counter

class CharacterBasedEncoder:
    
    def __init__(self, max_features=3000, pad='<pad>', unk='<unk>'):
        self.pad = pad
        self.unk = unk
        self.max_features=max_features
        
    def fit(self, text_set):
        self.counter = Counter()
        
        for text in text_set:
            self.counter.update(list(text))
        
        self.vocab_size = min(self.max_features, len(self.counter)) - 2
        words = self.counter.most_common(self.vocab_size)
        words.insert(0, self.pad)
        words.insert(1, self.unk)
        
        self.vocab = {c[0]: i for i, c in words}
    
    def encode(self, text, max_seq_len):
        e = [self.vocab.get(c, 1) for c in text]
        if len(e) <= max_seq_len:
            e.extend([0] * (max_seq_len - len(e)))
        if len(e) >= max_seq_len:
            e = e[:max_seq_len]
        
        return e

In [38]:
chars2idx = json.load(open("../sentiment/model/vocab.json"))
label2vec = {
    'question': [1, 0],
    'review': [0, 1]
}
SEQ_LEN = 256

def encode(text, y):
    text = list(text)
    encoded = [chars2idx.get(c, 0) for c in text]
    if len(encoded) < SEQ_LEN:
        encoded += [0] * (SEQ_LEN - len(encoded))
    if len(encoded) > SEQ_LEN:
        encoded = encoded[:SEQ_LEN]
    encoded.insert(0, label2vec[y])
    return encoded

In [39]:
encode_list = [encode(x[0], x[1]) for x in sample.itertuples(index=False)]
encode_list[0]

[[1, 0],
 56,
 189,
 1,
 60,
 80,
 169,
 339,
 425,
 4,
 1,
 89,
 133,
 244,
 87,
 32,
 1,
 76,
 289,
 456,
 811,
 126,
 23,
 1,
 314,
 456,
 811,
 50,
 12,
 430,
 23,
 1,
 38,
 800,
 10,
 4,
 10,
 238,
 9,
 1,
 24,
 101,
 1,
 178,
 349,
 83,
 4,
 48,
 1,
 89,
 133,
 244,
 87,
 1,
 166,
 42,
 11,
 1,
 16,
 401,
 11,
 4,
 48,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [41]:
train = pd.DataFrame(encode_list)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,"[1, 0]",56,189,1,60,80,169,339,425,4,...,0,0,0,0,0,0,0,0,0,0
1,"[1, 0]",42,1,503,64,212,9,19,33,1,...,0,0,0,0,0,0,0,0,0,0
2,"[1, 0]",10,216,364,87,99,69,1,102,137,...,0,0,0,0,0,0,0,0,0,0
3,"[1, 0]",108,42,68,1,8,7,11,1,167,...,0,0,0,0,0,0,0,0,0,0
4,"[0, 1]",87,135,8,5,1,175,22,5,1,...,0,0,0,0,0,0,0,0,0,0


In [42]:
len(train)

2610